In [1]:
!pip install pyspark
!pip install install-jdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0887fb97dab832000e702c39a71029096aa74c72b4b684265e4b42b57a9aedfc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, IDF
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

#membuat session
appName = 'Automated Scoring Using pySpark'
spark = SparkSession \
        .builder \
        .appName(appName) \
        .config('spark.some.config.option','some-value') \
        .getOrCreate()

In [3]:
import pandas as pd
auto_score = pd.read_excel("/content/drive/MyDrive/training_data_essay.xlsx")
auto_score

,npm,nama_peserta,jawaban,soal,skor_per_soal
0,0,Admin,"Tidak, Hanya membutuhkan satu karena satu soft...",1,100.00
1,0,Admin,Biaya dihitung berdasarkan waktu pengerjaan da...,2,100.00
2,0,Admin,Hak cipta adalah hak eksklusif bagi pencipta a...,3,100.00
3,0,Admin,Dijelaskan kepada klien jika huruf terlalu bes...,4,100.00
4,0,Admin,1. Melindungi dan menjamin pekerja atas hak ke...,5,100.00
...,...,...,...,...,...
115,1121020035,BS,posisi kepala dan leher harus tegak lurus deng...,8,48.41
116,1121020035,BS,kecocokan software camera dengan system hardwa...,9,65.89
117,1121020035,BS,okus dan exposure dipertimbangkan dalam operas...,10,48.51
118,1121020035,BS,peralatan yang dibutuhkan untuk transfer image...,11,78.41


In [4]:
df = spark.createDataFrame(auto_score)
df = df.select('soal', 'jawaban', 'skor_per_soal')
df.show()


+----+--------------------+-------------+
|soal|             jawaban|skor_per_soal|
+----+--------------------+-------------+
|   1|Tidak, Hanya memb...|        100.0|
|   2|Biaya dihitung be...|        100.0|
|   3|Hak cipta adalah ...|        100.0|
|   4|Dijelaskan kepada...|        100.0|
|   5|1. Melindungi dan...|        100.0|
|   6|Ruang Komputer, P...|        100.0|
|   7|Aturlah posisi pe...|        100.0|
|   8|Posisi Kepala dan...|        100.0|
|   9|1. Kecocokan soft...|        100.0|
|  10|1. Fokus dan expo...|        100.0|
|  11|1. Peralatan yang...|        100.0|
|  12|1. Dibuat grafik ...|        100.0|
|   1|tidak, cuma mengi...|         52.7|
|   2|biaya dihitung be...|        42.86|
|   3|hak membuat merup...|        42.16|
|   4|dipaparkan pada k...|        27.19|
|   5|1. mencegah serta...|        44.14|
|   6|ruang komputer, p...|        100.0|
|   7|aturlah posisi fi...|        57.68|
|   8|posisi kepala ser...|        45.71|
+----+--------------------+-------

In [5]:
regex_pattern = '[^a-zA-Z\\s]+|\\d+'
cleanningTrain = df.withColumn('jawaban_v2', regexp_replace('jawaban', regex_pattern,''))
cleanningTrain.show()

+----+--------------------+-------------+--------------------+
|soal|             jawaban|skor_per_soal|          jawaban_v2|
+----+--------------------+-------------+--------------------+
|   1|Tidak, Hanya memb...|        100.0|Tidak Hanya membu...|
|   2|Biaya dihitung be...|        100.0|Biaya dihitung be...|
|   3|Hak cipta adalah ...|        100.0|Hak cipta adalah ...|
|   4|Dijelaskan kepada...|        100.0|Dijelaskan kepada...|
|   5|1. Melindungi dan...|        100.0| Melindungi dan m...|
|   6|Ruang Komputer, P...|        100.0|Ruang Komputer Po...|
|   7|Aturlah posisi pe...|        100.0|Aturlah posisi pe...|
|   8|Posisi Kepala dan...|        100.0|Posisi Kepala dan...|
|   9|1. Kecocokan soft...|        100.0| Kecocokan softwa...|
|  10|1. Fokus dan expo...|        100.0| Fokus dan exposu...|
|  11|1. Peralatan yang...|        100.0| Peralatan yang d...|
|  12|1. Dibuat grafik ...|        100.0| Dibuat grafik ya...|
|   1|tidak, cuma mengi...|         52.7|tidak cuma men

In [6]:
tokenizer = Tokenizer(inputCol = 'jawaban_v2', outputCol = 'Tokenize_jawaban')
tokenizedTrain = tokenizer.transform(cleanningTrain)
tokenizedTrain.show()

+----+--------------------+-------------+--------------------+--------------------+
|soal|             jawaban|skor_per_soal|          jawaban_v2|    Tokenize_jawaban|
+----+--------------------+-------------+--------------------+--------------------+
|   1|Tidak, Hanya memb...|        100.0|Tidak Hanya membu...|[tidak, hanya, me...|
|   2|Biaya dihitung be...|        100.0|Biaya dihitung be...|[biaya, dihitung,...|
|   3|Hak cipta adalah ...|        100.0|Hak cipta adalah ...|[hak, cipta, adal...|
|   4|Dijelaskan kepada...|        100.0|Dijelaskan kepada...|[dijelaskan, kepa...|
|   5|1. Melindungi dan...|        100.0| Melindungi dan m...|[, melindungi, da...|
|   6|Ruang Komputer, P...|        100.0|Ruang Komputer Po...|[ruang, komputer,...|
|   7|Aturlah posisi pe...|        100.0|Aturlah posisi pe...|[aturlah, posisi,...|
|   8|Posisi Kepala dan...|        100.0|Posisi Kepala dan...|[posisi, kepala, ...|
|   9|1. Kecocokan soft...|        100.0| Kecocokan softwa...|[, kecocokan, 

In [7]:
from pyspark.ml.feature import StopWordsRemover

# Read the stopwords from the file
with open('/content/drive/MyDrive/id.stopwords.02.01.2016.txt') as f:
    stopwords = f.read().splitlines()

# Initialize the StopWordsRemover without the stopwords
stopword_remover = StopWordsRemover(inputCol='Tokenize_jawaban', outputCol='kata_bermakna')

# Set the custom stopwords
stopword_remover = stopword_remover.setStopWords(stopwords)

# Apply the stopword remover to the DataFrame
SwRemovedTrain = stopword_remover.transform(tokenizedTrain)
SwRemovedTrain.show()


+----+--------------------+-------------+--------------------+--------------------+--------------------+
|soal|             jawaban|skor_per_soal|          jawaban_v2|    Tokenize_jawaban|       kata_bermakna|
+----+--------------------+-------------+--------------------+--------------------+--------------------+
|   1|Tidak, Hanya memb...|        100.0|Tidak Hanya membu...|[tidak, hanya, me...|[membutuhkan, sof...|
|   2|Biaya dihitung be...|        100.0|Biaya dihitung be...|[biaya, dihitung,...|[biaya, dihitung,...|
|   3|Hak cipta adalah ...|        100.0|Hak cipta adalah ...|[hak, cipta, adal...|[hak, cipta, hak,...|
|   4|Dijelaskan kepada...|        100.0|Dijelaskan kepada...|[dijelaskan, kepa...|[klien, huruf, me...|
|   5|1. Melindungi dan...|        100.0| Melindungi dan m...|[, melindungi, da...|[, melindungi, me...|
|   6|Ruang Komputer, P...|        100.0|Ruang Komputer Po...|[ruang, komputer,...|[ruang, komputer,...|
|   7|Aturlah posisi pe...|        100.0|Aturlah posisi

In [8]:
hash = SwRemovedTrain.withColumn('hash_value', hash('kata_bermakna'))
hash.show()

+----+--------------------+-------------+--------------------+--------------------+--------------------+-----------+
|soal|             jawaban|skor_per_soal|          jawaban_v2|    Tokenize_jawaban|       kata_bermakna| hash_value|
+----+--------------------+-------------+--------------------+--------------------+--------------------+-----------+
|   1|Tidak, Hanya memb...|        100.0|Tidak Hanya membu...|[tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   2|Biaya dihitung be...|        100.0|Biaya dihitung be...|[biaya, dihitung,...|[biaya, dihitung,...|  868079823|
|   3|Hak cipta adalah ...|        100.0|Hak cipta adalah ...|[hak, cipta, adal...|[hak, cipta, hak,...| 1708817134|
|   4|Dijelaskan kepada...|        100.0|Dijelaskan kepada...|[dijelaskan, kepa...|[klien, huruf, me...| 1377676868|
|   5|1. Melindungi dan...|        100.0| Melindungi dan m...|[, melindungi, da...|[, melindungi, me...| 1706744332|
|   6|Ruang Komputer, P...|        100.0|Ruang Komputer Po...|[r

In [9]:
split = hash.randomSplit([0.7, 0.3])
train = split[0].withColumnRenamed('skor_per_soal', 'skor')
test = split[1].withColumnRenamed('skor_per_soal', 'true_skor')

In [10]:
row_train = train.count()
row_test = test.count()
print('data training', row_train)
print('data testing', row_test)

data training 82
data testing 38


In [11]:
train.show()


+----+--------------------+-----+--------------------+--------------------+--------------------+-----------+
|soal|             jawaban| skor|          jawaban_v2|    Tokenize_jawaban|       kata_bermakna| hash_value|
+----+--------------------+-----+--------------------+--------------------+--------------------+-----------+
|   1|tidak, cuma mengi...| 52.7|tidak cuma mengin...|[tidak, cuma, men...|[aplikasi, cocok,...|   53244348|
|   1|tidak, hanya memb...|100.0|tidak hanya membu...|[tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   1|tidak, hanya memb...|100.0|tidak hanya membu...|[tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   1|tidak, hanya memb...|100.0|tidak hanya membu...|[tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   2|Biaya dihitung be...|100.0|Biaya dihitung be...|[biaya, dihitung,...|[biaya, dihitung,...|  868079823|
|   2|biaya dihitung be...|100.0|biaya dihitung be...|[biaya, dihitung,...|[biaya, dihitung,...|  868079823|
|   2|biaya dihitun

In [12]:
test.show()

+----+--------------------+---------+--------------------+--------------------+--------------------+-----------+
|soal|             jawaban|true_skor|          jawaban_v2|    Tokenize_jawaban|       kata_bermakna| hash_value|
+----+--------------------+---------+--------------------+--------------------+--------------------+-----------+
|   1|Tidak, Hanya memb...|    100.0|Tidak Hanya membu...|[tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   2|biaya dihitung be...|    100.0|biaya dihitung be...|[biaya, dihitung,...|[biaya, dihitung,...|  868079823|
|   3|hak cipta adalah ...|    91.71|hak cipta adalah ...|[hak, cipta, adal...|[hak, cipta, hak,...| -348474399|
|   4|dijelaskan kepada...|    72.06|dijelaskan kepada...|[dijelaskan, kepa...|[klien, huruf, me...|  636521394|
|   4|dijelaskan kepada...|    72.06|dijelaskan kepada...|[dijelaskan, kepa...|[klien, huruf, me...|  636521394|
|   5|1. mencegah serta...|    44.14| mencegah serta m...|[, mencegah, sert...|[, mencegah, menj

In [13]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = 'hash_value', itemCol = 'soal', ratingCol = 'skor')
model = als.fit(train)

In [14]:
prediction = model.transform(test)

In [15]:
prediction.select('soal', 'hash_value', 'true_skor', 'prediction').show()

+----+-----------+---------+----------+
|soal| hash_value|true_skor|prediction|
+----+-----------+---------+----------+
|  12|-1557371155|    64.29|  64.28996|
|  12|  481493723|    100.0|       NaN|
|   1|-1678823266|    100.0|  99.99991|
|   1|-1678823266|    100.0|  99.99991|
|   6|  829849610|    100.0|  99.99992|
|   3| -348474399|    91.71|  91.70997|
|   5|-1211017882|    90.27|       NaN|
|   9|-2081253499|    100.0|       NaN|
|   9| -720994262|    84.88|  84.87993|
|   4|  636521394|    72.06|  72.63663|
|   4|  636521394|    72.06|  72.63663|
|   4|  636521394|    72.06|  72.63663|
|   8|-1861440661|    100.0|  99.99996|
|   7|-1894676254|    86.22|  86.21996|
|   7|-1894676254|    86.22|  86.21996|
|   7|-1894676254|    86.22|  86.21996|
|   7|  624824635|    59.87|       NaN|
|  10|-2119091795|    100.0|       NaN|
|  11|-1374699892|    77.63|       NaN|
|  11| -521292198|    100.0|       NaN|
+----+-----------+---------+----------+
only showing top 20 rows



In [16]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='true_skor', predictionCol='prediction')
rmse = evaluator.evaluate(prediction)
print('Root Mean Square Error (RMSE):', rmse)


Root Mean Square Error (RMSE): nan


In [17]:

evaluator = RegressionEvaluator(metricName='rmse', labelCol='true_skor', predictionCol='prediction')

# Evaluate the predictions
mse = evaluator.evaluate(prediction)

# Print the RMSE
print(' Mean Square Error (RMSE):', mse)


 Mean Square Error (RMSE): nan


In [18]:
prediction.count()
a = prediction.count()
print('sebelum missing value dihapus:', a,'baris')
cleanPred = prediction.dropna(how = 'any', subset = ['prediction'])
b = cleanPred.count()
print('setelah missing value dihapus:', b, 'baris')
print('missing value:', a-b, 'baris')

sebelum missing value dihapus: 38 baris
setelah missing value dihapus: 26 baris
missing value: 12 baris


In [19]:
mse = evaluator.evaluate(cleanPred)

# Print the MSE
print('Mean Squared Error (MSE):', mse)

Mean Squared Error (MSE): 0.19587052775507344


In [20]:
rmse = evaluator.evaluate(cleanPred)
print('root mean squard error:', rmse)

root mean squard error: 0.19587052775507344
